In [ ]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split

In [ ]:
ids_and_labels_path = "/data/raw/crowdbreaks_data/crowdbreaks_tweet_ids_and_labels.csv"
tweets_path = "/data/raw/crowdbreaks_data/crowdbreaks_tweets.csv"
dataframes_path = "/data/processed/dataframes"

In [ ]:
ids_and_labels = pd.read_csv(ids_and_labels_path)
tweets = pd.read_csv(tweets_path)

In [ ]:
tweet_and_label = ids_and_labels.join(tweets.rename(columns={'id':'tweet_id'}).set_index('tweet_id'), on='tweet_id', rsuffix='_')
tweet_and_label = tweet_and_label.dropna().drop(columns=['label_'])
tweet_and_label = tweet_and_label[tweet_and_label['agreement'] >= 0.66].reset_index(drop=True)

In [ ]:
mapping = {0:0, 1:1, -1:2}
tweet_and_label['label'] = tweet_and_label['label'].map(mapping)

In [ ]:
agreement1 = tweet_and_label[tweet_and_label['agreement'] == 1.0].reset_index(drop=True)
agreement66 = tweet_and_label[(tweet_and_label['agreement'] >= 0.66) & (tweet_and_label['agreement'] < 1.0)].reset_index(drop=True)

### Data Preparation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(agreement1['text'].values,
                                                    agreement1['label'].astype('int').values,
                                                    test_size=0.2, random_state=42,
                                                    stratify = agreement1['label'].astype('int').values)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=0.5, random_state=42,
                                                    stratify = y_test)

In [ ]:
X_train = np.concatenate([X_train, agreement66['text'].values])
y_train = np.concatenate([y_train, agreement66['label'].values])

In [ ]:
df_train = pd.DataFrame(np.concatenate([X_train.reshape((-1, 1)), y_train.reshape((-1, 1))], axis=1),
                                      columns=['text', 'labels'])
df_test = pd.DataFrame(np.concatenate([X_test.reshape((-1, 1)), y_test.reshape((-1, 1))], axis=1),
                                      columns=['text', 'labels'])
df_val = pd.DataFrame(np.concatenate([X_val.reshape((-1, 1)), y_val.reshape((-1, 1))], axis=1),
                                      columns=['text', 'labels'])

### Loading Model

In [ ]:
with open("/models/sentiment_models/best_model.db", 'rb') as f:
    model = pickle.load(f)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
predictions, raw_outputs = model.predict(df_test['text'].tolist())
probas_pred = np.array(tf.nn.softmax(raw_outputs))

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.85      0.91       571
           1       0.82      0.96      0.88       397
           2       0.77      0.74      0.75        62

    accuracy                           0.89      1030
   macro avg       0.85      0.85      0.85      1030
weighted avg       0.90      0.89      0.89      1030



### Preparing Labels

In [ ]:
zero_vs_rest_prob = probas_pred[:, 0]
one_vs_rest_prob = probas_pred[:, 1]
two_vs_rest_prob = probas_pred[:, 2]

zero_vs_rest = np.where(predictions == 0, 1, 0)
one_vs_rest = np.where(predictions == 1, 1, 0)
two_vs_rest = np.where(predictions == 2, 1, 0)

zero_vs_rest_true = np.where(y_test == 0, 1, 0)
one_vs_rest_true = np.where(y_test == 1, 1, 0)
two_vs_rest_true = np.where(y_test == 2, 1, 0)

In [ ]:
print(classification_report(zero_vs_rest_true, zero_vs_rest))
print(classification_report(one_vs_rest_true, one_vs_rest))
print(classification_report(two_vs_rest_true, two_vs_rest))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       459
           1       0.97      0.85      0.91       571

    accuracy                           0.90      1030
   macro avg       0.90      0.91      0.90      1030
weighted avg       0.91      0.90      0.90      1030

              precision    recall  f1-score   support

           0       0.97      0.87      0.91       633
           1       0.82      0.96      0.88       397

    accuracy                           0.90      1030
   macro avg       0.89      0.91      0.90      1030
weighted avg       0.91      0.90      0.90      1030

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       968
           1       0.77      0.74      0.75        62

    accuracy                           0.97      1030
   macro avg       0.88      0.86      0.87      1030
weighted avg       0.97      0.97      0.97      1030



### precision_recall curve



In [ ]:
precision_0, recall_0, thresholds_0 = precision_recall_curve(zero_vs_rest_true, zero_vs_rest_prob)
precision_1, recall_1, thresholds_1 = precision_recall_curve(one_vs_rest_true, one_vs_rest_prob)
precision_2, recall_2, thresholds_2 = precision_recall_curve(two_vs_rest_true, two_vs_rest_prob)

zero_auc = auc(recall_0, precision_0)
one_auc = auc(recall_1, precision_1)
two_auc = auc(recall_2, precision_2)

thresholds_0 = np.array([0] + thresholds_0.tolist())
thresholds_1 = np.array([0] + thresholds_1.tolist())
thresholds_2 = np.array([0] + thresholds_2.tolist())

In [ ]:
zero_auc_th = auc(thresholds_0, precision_0)
one_auc_th = auc(thresholds_1, precision_1)
two_auc_th = auc(thresholds_2, precision_2)

In [ ]:
threshes = [thresholds_0, thresholds_1, thresholds_2]
idxs = [np.arange(threshes[idx].shape[0])[threshes[idx] >= 0.99][0] for idx in range(3)]

In [ ]:
precs_and_recs = [[precision_0[idxs[0]], recall_0[idxs[0]]], [precision_1[idxs[1]], recall_1[idxs[1]]], [precision_2[idxs[2]], recall_2[idxs[2]]]]

In [ ]:
result_dict = {'recall': {'neutral': recall_0, 'positive': recall_1, 'negative': recall_2},
               'precision': {'neutral': precision_0, 'positive': precision_1, 'negative': precision_2},
               'threshold': {'neutral': thresholds_0, 'positive': thresholds_1, 'negative': thresholds_2},
               'precs_and_recs_auc': {'neutral': zero_auc, 'positive': one_auc, 'negative': two_auc},
               'precs_and_recs': precs_and_recs,
               'th_auc': {'neutral': zero_auc_th, 'positive': one_auc_th, 'negative': two_auc_th}}

with open(f"{dataframes_path}/precision_recall_threshold_curve_results.pkl", 'wb') as f:
    pickle.dump(result_dict, f, protocol=4)